In [9]:
import math
import random
import json

# Constants for simulation
OVERLAY_RADIUS = 100         # in meters: radius of true/perturbed circle
POKEMON_MAX_RADIUS = 200     # in meters: radius around fixed center to generate Pokémon
POKEMON_SPACING = 50         # in meters: spacing for generating Pokémon positions

def load_noise_pairs(filename):
    """
    Load noise pairs from a JSON file.
    The JSON file is expected to contain an array of pairs, e.g.:
      [[noise_x1, noise_y1], [noise_x2, noise_y2], ...]
    """
    with open(filename, 'r') as f:
        noise_data = json.load(f)
    # Return as list of (noise_x, noise_y) tuples
    return [(pair[0], pair[1]) for pair in noise_data]

def distance_meters(lat1, lon1, lat2, lon2):
    """
    Approximate the distance between two lat/lon points in meters using an equirectangular approximation.
    This is sufficient for short distances.
    """
    # Average latitude in radians for longitude scaling
    avg_lat = math.radians((lat1 + lat2) / 2)
    # Convert differences to meters (1 degree latitude ~111320 m)
    dx = (lon2 - lon1) * (111320 * math.cos(avg_lat))
    dy = (lat2 - lat1) * 111320
    return math.sqrt(dx * dx + dy * dy)

def add_noise_to_location(lat, lon, noise_x, noise_y):
    """
    Given a true location (lat, lon) and noise in meters, convert the noise to degrees and add it.
    """
    # 1 degree latitude is approximately 111320 meters.
    delta_lat = noise_y / 111320.0
    # For longitude, adjust by the cosine of the latitude.
    delta_lon = noise_x / (111320.0 * math.cos(math.radians(lat)))
    return lat + delta_lat, lon + delta_lon

def generate_uniform_pokemon_positions(center, max_radius, spacing):
    """
    Generate a list of (lat, lon) positions uniformly within a circle (center, max_radius).
    This follows a grid-like approach similar to your Kotlin code.
    """
    center_lat, center_lon = center
    positions = []
    # Conversion factors from meters to degrees (latitude constant, longitude depends on latitude)
    meter_to_deg_lat = 1.0 / 111320.0
    meter_to_deg_lon = 1.0 / (111320.0 * math.cos(math.radians(center_lat)))
    
    radius_deg_lat = max_radius * meter_to_deg_lat
    radius_deg_lon = max_radius * meter_to_deg_lon
    min_lat = center_lat - radius_deg_lat
    max_lat = center_lat + radius_deg_lat
    min_lon = center_lon - radius_deg_lon
    max_lon = center_lon + radius_deg_lon
    spacing_deg_lat = spacing * meter_to_deg_lat
    spacing_deg_lon = spacing * meter_to_deg_lon

    lat = min_lat
    while lat <= max_lat:
        lon = min_lon
        while lon <= max_lon:
            # Check if this candidate lies within the max_radius circle
            if distance_meters(center_lat, center_lon, lat, lon) <= max_radius:
                positions.append((lat, lon))
            lon += spacing_deg_lon
        lat += spacing_deg_lat
    return positions

def simulate_show_rate(true_location, noise_pairs, pokemon_positions, iterations=100):
    """
    For a given true location, simulate multiple perturbed locations by randomly picking noise pairs.
    For each perturbed sample, compute the show rate as:
       (number of Pokémon in both true and perturbed circle) / (number in true circle) * 100
    Returns the average show rate over the specified number of iterations.
    """
    true_lat, true_lon = true_location
    rates = []
    for _ in range(iterations):
        # Pick a random noise pair and compute the perturbed location.
        noise_x, noise_y = random.choice(noise_pairs)
        perturbed_lat, perturbed_lon = add_noise_to_location(true_lat, true_lon, noise_x, noise_y)
        
        count_true = 0
        count_intersection = 0
        
        # For each Pokémon, check if it lies within the true circle and perturbed circle.
        for p_lat, p_lon in pokemon_positions:
            if distance_meters(true_lat, true_lon, p_lat, p_lon) <= OVERLAY_RADIUS:
                count_true += 1
                if distance_meters(perturbed_lat, perturbed_lon, p_lat, p_lon) <= OVERLAY_RADIUS:
                    count_intersection += 1
        # Calculate the show rate for this iteration.
        show_rate = (count_intersection / count_true) * 100 if count_true > 0 else 0
        rates.append(show_rate)
    return sum(rates) / len(rates)

def simulate_for_mechanism(noise_filename, true_locations, fixed_pokemon_center, iterations_per_location=100):
    """
    Run the simulation for a given mechanism (specified via its noise file).
    For each true location, the function computes the average show rate over the given iterations.
    Then it returns the overall average across all true locations.
    """
    noise_pairs = load_noise_pairs(noise_filename)
    # Generate Pokémon positions once (static anchors) around the fixed center.
    pokemon_positions = generate_uniform_pokemon_positions(fixed_pokemon_center, POKEMON_MAX_RADIUS, POKEMON_SPACING)
    location_rates = []
    for loc in true_locations:
        rate = simulate_show_rate(loc, noise_pairs, pokemon_positions, iterations=iterations_per_location)
        location_rates.append(rate)
        print(f"True location {loc} average show rate: {rate:.2f}%")
    overall_average = sum(location_rates) / len(location_rates) if location_rates else 0
    return overall_average

def main():
    # Fixed Pokémon generation center (e.g., RIT, Rochester, NY)
    fixed_pokemon_center = (43.083789, -77.680391)
    
    # Define several true locations (you can adjust these as needed)
    true_locations = [
        (43.083789, -77.680391),  # Location A
        (43.083900, -77.680500),  # Location B
        (43.083500, -77.680200),  # Location C
        (43.083850, -77.680450),  # Location D
        (43.083600, -77.680600)   # Location E
    ]
    
    # Define noise file names for the two mechanisms.
    laplace_noise_file = r"C:\Users\ss6365\AndroidStudioProjects\testformap2\app\src\main\assets\noise_laplace_high.json"
    staircase_noise_file = r"C:\Users\ss6365\AndroidStudioProjects\testformap2\app\src\main\assets\noise_staircase_high.json"
    
    print("Simulating for Laplace mechanism:")
    avg_show_rate_laplace = simulate_for_mechanism(
        laplace_noise_file, true_locations, fixed_pokemon_center, iterations_per_location=100
    )
    print(f"Overall average show rate for Laplace: {avg_show_rate_laplace:.2f}%\n")
    
    print("Simulating for Staircase mechanism:")
    avg_show_rate_staircase = simulate_for_mechanism(
        staircase_noise_file, true_locations, fixed_pokemon_center, iterations_per_location=100
    )
    print(f"Overall average show rate for Staircase: {avg_show_rate_staircase:.2f}%")

if __name__ == "__main__":
    main()


Simulating for Laplace mechanism:
True location (43.083789, -77.680391) average show rate: 92.80%
True location (43.0839, -77.6805) average show rate: 86.54%
True location (43.0835, -77.6802) average show rate: 88.85%
True location (43.08385, -77.68045) average show rate: 90.73%
True location (43.0836, -77.6806) average show rate: 88.85%
Overall average show rate for Laplace: 89.55%

Simulating for Staircase mechanism:
True location (43.083789, -77.680391) average show rate: 94.50%
True location (43.0839, -77.6805) average show rate: 92.08%
True location (43.0835, -77.6802) average show rate: 96.15%
True location (43.08385, -77.68045) average show rate: 96.91%
True location (43.0836, -77.6806) average show rate: 94.77%
Overall average show rate for Staircase: 94.88%
